# Ames Housing Prices
Eric Hedgren

# Stage 2: Get and Prepare the Data

## Importing data and needed libraries

Link to Data:
https://www.kaggle.com/datasets/marcopale/housing/code

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv('data/AmesHousing.csv')
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2930 non-null   int64  
 1   PID              2930 non-null   int64  
 2   MS SubClass      2930 non-null   int64  
 3   MS Zoning        2930 non-null   object 
 4   Lot Frontage     2440 non-null   float64
 5   Lot Area         2930 non-null   int64  
 6   Street           2930 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2930 non-null   object 
 9   Land Contour     2930 non-null   object 
 10  Utilities        2930 non-null   object 
 11  Lot Config       2930 non-null   object 
 12  Land Slope       2930 non-null   object 
 13  Neighborhood     2930 non-null   object 
 14  Condition 1      2930 non-null   object 
 15  Condition 2      2930 non-null   object 
 16  Bldg Type        2930 non-null   object 
 17  House Style   

## Finding outliers
Finding outliers with SalePrice and the 2 highest correlations with SalePrice: Overall Qual and Gr Liv Area

In [9]:
df.corrwith(df['SalePrice']).sort_values(ascending=False)

ValueError: could not convert string to float: 'RL'

In [2]:
sns.scatterplot(x='Overall Qual', y='SalePrice', data=df)

NameError: name 'sns' is not defined

In [ ]:
sns.scatterplot(x='Gr Liv Area', y='SalePrice', data=df)

In [ ]:
df = df.drop_duplicates()
drop_outliers = df[(df['Gr Liv Area'] > 4000) & (df['SalePrice'] < 400000)].index
df = df.drop(drop_outliers, axis=0)

In [ ]:
sns.scatterplot(x='Overall Qual', y='SalePrice', data=df)

In [ ]:
sns.scatterplot(x='Gr Liv Area', y='SalePrice', data=df)

## Missing Data

In [ ]:
df = df.drop('PID', axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
def percent_missing(df):
  percent_nan = 100 * df.isnull().sum() / len(df)
  percent_nan = percent_nan[percent_nan>0].sort_values()
  return percent_nan

percent_nan = percent_missing(df)

In [ ]:
plt.figure(figsize=(12, 6))
percent_plot = sns.barplot(x=percent_nan.index, y=percent_nan,
                           palette='ch:s=.25,rot=-.25')
percent_plot.set_xticklabels(percent_plot.get_xticklabels(), rotation=45,
                      horizontalalignment='right')
plt.show()

## Dropping Rows/Columns and filling in Columns with missing values

Drop rows if there are few missing values in a column.
Drop columns if there are too many missing vales.
Fill in the remaining missing vales with 0 or none because if the value is nan then the house does not have that feature.

In [ ]:
df_object_missing = []
df_int_missing = []
dump = []

for c in df:
  dtype = df[c].dtypes
  if (c in df and c in percent_nan and percent_nan[c] < 50):
    if (dtype == 'O'):
      df_object_missing.append(c)
    else:
      df_int_missing.append(c)
  elif (c in df and c in percent_nan and percent_nan[c] >= 50):
    dump.append(c)

In [ ]:
df = df.dropna(axis=0, subset=['Electrical', 'Garage Cars'])

In [ ]:
df = df.drop(dump, axis=1)

In [ ]:
df[df_object_missing] = df[df_object_missing].fillna('None')
df[df_int_missing] = df[df_int_missing].fillna(0)

## Turning Categorical Data into Dummy Variables
Creating dummy variables and replacing the categorical data with them so that the linear regression model and use them. Then replacing the columns of the dummied columns with the dummy variables.

In [ ]:
df = pd.get_dummies(df, drop_first=True)
df.shape

# Stage 3: Explore the data

## Finding Correlations

In [ ]:
corr = df.corrwith(df['SalePrice']).sort_values(ascending=False)
corr

In [ ]:
a = []
b = []
c = []
columns = corr.index

# filling list up with the values based on if they are over under or between
# 0.25 and -0.25
for i in range(0, len(corr)):
  if (corr[i] > 0.25):
    a.append(columns[i])
  elif (corr[i] < -0.25):
    c.append(columns[i])
  else:
    b.append(columns[i])

# Filling lists up with nan until they are the same size
while (len(a) != len(b)):
  a.append(np.nan)
while (len(c) != len(b)):
  c.append(np.nan)

d = {'Correlated (>0.25)' : a, 'Not Correlated' : b,
     'Inversely Correlated (>0.25)' : c}

corr_lvl = pd.DataFrame(d)

In [ ]:
plt.figure(figsize=(15,10))
plt.title('Percentage of Correlated Features to SalePrice')
colors = sns.color_palette('bright')
plt.pie(corr_lvl.count(), labels=corr_lvl.columns, colors=colors,
        autopct='%.0f%%')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('Correlation of the top 50 features to SalePrice')
corr_plot = sns.barplot(x=corr[1:50].index, y=corr[1:50],
                        palette='mako')
corr_plot.set_xticklabels(corr_plot.get_xticklabels(), rotation=90,
                      horizontalalignment='right')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('Correlation of the bottom 50 features to SalePrice')
corr_plot = sns.barplot(x=corr[210:].index, y=corr[210:],
                        palette='crest')
corr_plot.set_xticklabels(corr_plot.get_xticklabels(), rotation=90,
                      horizontalalignment='right')
plt.show()

In [ ]:
df.boxplot(column='SalePrice', by='Overall Qual', figsize=(10,7), grid=False)

Removing features because they have little to no correlation with SalePrice.


In [ ]:
df = df.drop(corr_lvl['Not Correlated'], axis=1)
df.shape

In [ ]:
df.corrwith(df['SalePrice']).sort_values(ascending=False)

In [ ]:
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook, save, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot

output_notebook()

# add graphs with bokeh to show correlation
# could use scatter plot to show different SF compared to SalePrice

In [ ]:
y = 'SalePrice'

# gridplot 1
x = 'Gr Liv Area'
g1 = figure(title='Gr Liv Area by SalePrice', x_axis_label=x,
            y_axis_label=y)
g1.diamond(x=x, y=y, source=df, color='blue', size=10, alpha=0.6)

# gridplot 2
x = 'Total Bsmt SF'
g2 = figure(title='Total Bsmt SF vs. SalePrice', x_axis_label=x,
            y_axis_label=y)
g2.hex(x=x, y=y, source=df, color='green', size=10, alpha=0.6)

# gridplot 3
x = '1st Flr SF'
g3 = figure(title='1st Flr SF vs. SalePrice', x_axis_label=x,
            y_axis_label=y)
g3.circle(x=x, y=y, source=df, color='red', size=10, alpha=0.6)

# gridplot 4
x = 'Garage Area'
g4 = figure(title='Garage Area vs. SalePrice', x_axis_label=x,
            y_axis_label=y)
g4.triangle(x=x, y=y, source=df, color='pink', size=10, alpha=0.6)

# gridplot 5
x = 'Mas Vnr Area'
g5 = figure(title='Mas Vnr Area vs. SalePrice', x_axis_label=x,
            y_axis_label=y)
g5.square(x=x, y=y, source=df, color='orange', size=10, alpha=0.6)

grid = gridplot([g1, g2, g3, g4, g5], ncols=3, plot_width=500, plot_height=500)
show(grid)

# Stage 4: Model the Data

## Splitting the Data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = df[['SalePrice']]
X = df.drop('SalePrice', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

## Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

In [ ]:
lin_predictions = lin_reg.predict(X_test)

lin_reg_score = lin_reg.score(X_test, y_test)

## Ridge Regression Model

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge()

ridge.fit(X_train, y_train)

In [ ]:
ridge_predictions = ridge.predict(X_test)

ridge_score = ridge.score(X_test, y_test)

## Comparing Accuracy Scores

Linear Regression and Ridge Regression are very close in accuracy as shown below, but Linear Regression turned out to be more accurate by 0.0002%.

In [ ]:
print('Linear Regression:', lin_reg_score)
print('Ridge Regression:', ridge_score)